This notebook makes some visualizations, both static and interactive, with [Altair](https://altair-viz.github.io/).

Make sure Altair is installed:

```python
!pip install altair vega_datasets
```

In [3]:
import altair as alt
import string

## Static, just upper-case alphabetic characters (in paper)

In [4]:
# get upper case alpha characters
alphas = list(string.ascii_lowercase)
alphas_lower = alphas.copy()
# add larger ones
for a in alphas_lower:
    alphas.append(a.upper())

upperAlphas = alphas[26:]
print(upperAlphas)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [2]:
min_percent = 0.01 # only plot things that are at this level (in %) or above in terms of frequency in dataset
alphas_hist = ['A', 'K'] # letters to histogram

In [ ]:

hist_height = height // (len(alphas_hist)+1)

sort_pdf = np.unique(dfin['pdf_letters']).tolist()
sort_pdf.sort()
extra_ocr = []
for o in dfin['ocr_letters'].unique():
    if o not in sort_pdf:
        extra_ocr.append(o)

extra_ocr.sort()

sort_ocr = sort_pdf.copy()
sort_ocr.extend(extra_ocr)

length = 200
legendY = -40
#legendX = 100+int(length/100.*30)
legendX = 0 + width//2 - length//2

if log:
    color = alt.Color(percent_column+":Q", scale=alt.Scale(type='log',scheme=scheme, domain=[min_percent, 100]),
                      title=color_title, legend=alt.Legend(
                        orient='none',
                        legendX=legendX, legendY=legendY,
                        direction='horizontal',
                        titleAnchor='middle', gradientLength=length))
else:
    color = alt.Color(percent_column+"value:Q", scale=alt.Scale(scheme=scheme,domain=[min_percent, 100]),
                      title=color_title)

chart1 = alt.Chart(dfin).mark_rect().encode(
    alt.Y("pdf_letters:O",sort=sort_pdf,title=pdf_title),
    alt.X("ocr_letters:O",sort=sort_ocr,title=ocr_title),
    color=color
).properties(
    height=height,
    width=width
)
# ).configure_axis(
#     labelFontSize=20,
#     titleFontSize=20
# )

#chart = alt.vconcat(chart1, chart2, center=True)
chart_hists = []
for a in alphas_hist:
    dfin_larger = dfin.loc[dfin['pdf_letters'] == a]
    chart2 = alt.Chart(dfin_larger,title="GT Letter = " + a).mark_bar().transform_filter(
       alt.FieldRangePredicate(field=percent_column, range=[100, min_percent])
    ).encode(
        alt.X('ocr_letters:O', sort='-y',title=ocr_title),#,labelFontSize=hist_labelFontSize),
        alt.Y("% of all OCR tokens:Q", scale=alt.Scale(type='log',domain=[min_percent,100]), title='% Correct')
    ).properties(
        width=hist_width,
        height=hist_height
    )
    chart_hists.append(chart2)

chart_hists_all = alt.vconcat(*chart_hists, center=True)

#chart = chart1 | chart_hists_all
chart = alt.hconcat(chart1,chart_hists_all,center=True)
    
chart.configure_axis(
    labelFontSize=20,
    titleFontSize=20
)